In [1]:
from itertools import product
epsilons = [1,2,3]
deltas = [4,5,6]
print(list(product(epsilons, deltas)))

[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]


In [1]:
import sys
sys.path.append('/home/dbswldms316/FLamby/')


### 데이터 불러오기

In [2]:
from flamby.datasets.fed_camelyon16 import FedCamelyon16, Camelyon16Raw

# Raw dataset
mydataset_raw = Camelyon16Raw()

# Pooled test dataset (전체 데이터셋 - centralized 일 때!)
mydataset_pooled = FedCamelyon16(train=False, pooled=True)

# Center 2 train dataset
mydataset_local2= FedCamelyon16(center=2, train=True, pooled=False)

WARNING USING DEBUG MODE DATASET EVEN THOUGH DEBUG WAS SET TO FALSE, COULD NOT FIND NON DEBUG DATASET CONFIG FILE


ValueError: It seems the dataset fed_camelyon16 was not downloaded as the config file is not found for either normal or debug mode. Please refer to the download instructions inside FLamby/flamby/datasets/fed_camelyon16/README.md

### Local Training

In [2]:
from flamby.datasets.fed_isic2019.dataset import Isic2019Raw, FedIsic2019

In [3]:
mydataset_raw = Isic2019Raw()
mydataset_pooled = FedIsic2019(train=True, pooled=True)


In [4]:
# To load the first center as a pytorch dataset
center0 = FedIsic2019(center=0, train=True)
# To load the second center as a pytorch dataset
center1 = FedIsic2019(center=1, train=True)
# To load the 3rd center ...
center2 = FedIsic2019(center=2, train=True)
center3 = FedIsic2019(center=3, train=True)
center4 = FedIsic2019(center=4, train=True)
center5 = FedIsic2019(center=5, train=True)

In [4]:
from flamby.datasets.fed_isic2019 import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    Optimizer,
)



In [5]:
import torch
# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer
train_dataset = FedIsic2019(center=1, train=True, pooled=False)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
lossfunc = BaselineLoss()
model = Baseline()
optimizer = Optimizer(model.parameters(), lr=LR)

# Traditional pytorch training loop
for epoch in range(0, NUM_EPOCHS_POOLED):
    for idx, (X, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(X)
        loss = lossfunc(outputs, y)
        loss.backward()
        optimizer.step()
    

Loaded pretrained weights for efficientnet-b0
Number of features output by EfficientNet 1280


In [6]:
# Evaluation
from flamby.utils import evaluate_model_on_tests
# Instantiation of a list of the local test sets
test_dataloaders = [
            torch.utils.data.DataLoader(
                FedIsic2019(center=1, train=False, pooled=False),
                batch_size=BATCH_SIZE,
                shuffle=False,
                num_workers=0,
            )
            #for i in range(NUM_CLIENTS)
        ]
# Function performing the evaluation
dict_cindex = evaluate_model_on_tests(model, test_dataloaders, metric)
print(dict_cindex)


100%|██████████| 1/1 [00:03<00:00,  3.21s/it]

{'client_test_0': 0.7765621995770045}


결과 : {'client_test_0': 0.7558902342192252, 'client_test_1': 0.41120404195774307, 'client_test_2': 0.6664539082124209, 'client_test_3': 0.435638777683362, 'client_test_4': 0.3176664605236034, 'client_test_5': 0.5899024024024024}

### FL

In [3]:
import torch
from flamby.utils import evaluate_model_on_tests

# 2 lines of code to change to switch to another dataset
from flamby.datasets.fed_isic2019 import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    get_nb_max_rounds
)
from flamby.datasets.fed_isic2019 import FedIsic2019

# 1st line of code to change to switch to another strategy
from flamby.strategies.fed_avg import FedAvg as strat

# We loop on all the clients of the distributed dataset and instantiate associated data loaders
train_dataloaders = [
            torch.utils.data.DataLoader(
                FedIsic2019(center = i, train = True, pooled = False),
                batch_size = BATCH_SIZE,
                shuffle = True,
                num_workers = 0
            )
            for i in range(NUM_CLIENTS)
        ]

lossfunc = BaselineLoss()
m = Baseline()

# Federated Learning loop
# 2nd line of code to change to switch to another strategy (feed the FL strategy the right HPs)
args = {
            "training_dataloaders": train_dataloaders,
            "model": m,
            "loss": lossfunc,
            "optimizer_class": torch.optim.SGD,
            "learning_rate": LR / 10.0,
            "num_updates": 100,
# This helper function returns the number of rounds necessary to perform approximately as many
# epochs on each local dataset as with the pooled training
            "nrounds": get_nb_max_rounds(100),
        }
s = strat(**args)
m = s.run()[0]

Loaded pretrained weights for efficientnet-b0
Number of features output by EfficientNet 1280


100%|██████████| 9/9 [16:58<00:00, 113.22s/it]


In [4]:

# Evaluation
# We only instantiate one test set in this particular case: the pooled one
test_dataloaders = [
            torch.utils.data.DataLoader(
                FedIsic2019(center=i, train = False, pooled = True),
                batch_size = BATCH_SIZE,
                shuffle = False,
                num_workers = 0,
            )
            for i in range(NUM_CLIENTS)

        ]
dict_cindex = evaluate_model_on_tests(m, test_dataloaders, metric)
print(dict_cindex)

100%|██████████| 6/6 [00:34<00:00,  5.67s/it]

{'client_test_0': 0.2511727121732762, 'client_test_1': 0.2511727121732762, 'client_test_2': 0.2511727121732762, 'client_test_3': 0.2511727121732762, 'client_test_4': 0.2511727121732762, 'client_test_5': 0.2511727121732762}


결과 : {'client_test_0': 0.2784241354353424, 'client_test_1': 0.2784241354353424, 'client_test_2': 0.2784241354353424, 'client_test_3': 0.2784241354353424, 'client_test_4': 0.2784241354353424, 'client_test_5': 0.2784241354353424}



In [47]:
!pip3 install tensorboard

  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 13.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.5/186.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 26.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 33.8 MB/s eta 0:00:00
Using cached cac

## FedHeartDisease

In [4]:
from flamby.datasets.fed_heart_disease import FedHeartDisease, HeartDiseaseRaw

# To load the first center as a pytorch dataset
center0 = FedHeartDisease(center=0, train=True)
# To load the second center as a pytorch dataset
center1 = FedHeartDisease(center=1, train=True)
# To sample batches from each of the local datasets use the traditional pytorch API
from torch.utils.data import DataLoader as dl

X, y = iter(dl(center0, batch_size=16, shuffle=True, num_workers=0)).__next__()

In [11]:
import torch
from flamby.utils import evaluate_model_on_tests

# 2 lines of code to change to switch to another dataset
from flamby.datasets.fed_heart_disease import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    Optimizer,
)
from flamby.datasets.fed_heart_disease import FedHeartDisease as FedDataset

# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer
train_dataset = FedDataset(center=0, train=True, pooled=False)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
lossfunc = BaselineLoss()
model = Baseline()
optimizer = Optimizer(model.parameters(), lr=LR)

# Traditional pytorch training loop
for epoch in range(0, NUM_EPOCHS_POOLED):
    for idx, (X, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(X)
        loss = lossfunc(outputs, y)
        loss.backward()
        optimizer.step()

# Evaluation
# Instantiation of a list of the local test sets
test_dataloaders = [
            torch.utils.data.DataLoader(
                FedDataset(center=i, train=False, pooled=False),
                batch_size=BATCH_SIZE,
                shuffle=False,
                num_workers=0,
            )
            for i in range(NUM_CLIENTS)
        ]
# Function performing the evaluation
dict_cindex = evaluate_model_on_tests(model, test_dataloaders, metric)
print(dict_cindex)


100%|██████████| 4/4 [00:00<00:00, 399.91it/s]

{'client_test_0': 0.7788461538461539, 'client_test_1': 0.7752808988764045, 'client_test_2': 0.75, 'client_test_3': 0.6444444444444445}


## Fed-TCGA

In [5]:
from flamby.datasets.fed_tcga_brca import TcgaBrcaRaw, FedTcgaBrca

# Raw dataset
mydataset_raw = TcgaBrcaRaw()

# Pooled test dataset
mydataset_pooled = FedTcgaBrca(train=False, pooled=True)

# Center 2 train dataset
mydataset_local2= FedTcgaBrca(center=2, train=True, pooled=False)

Saving license agreement
You may now proceed to download.



### Local training

In [6]:
import torch
from flamby.utils import evaluate_model_on_tests

# 2 lines of code to change to switch to another dataset
from flamby.datasets.fed_tcga_brca import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    Optimizer,
)
from flamby.datasets.fed_tcga_brca import FedTcgaBrca as FedDataset

# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer
train_dataset = FedDataset(center=0, train=True, pooled=False)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
lossfunc = BaselineLoss()
model = Baseline()
optimizer = Optimizer(model.parameters(), lr=LR)

# Traditional pytorch training loop
for epoch in range(0, NUM_EPOCHS_POOLED):
    for idx, (X, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(X)
        loss = lossfunc(outputs, y)
        loss.backward()
        optimizer.step()

# Evaluation
# Instantiation of a list of the local test sets
test_dataloaders = [
            torch.utils.data.DataLoader(
                FedDataset(center=i, train=False, pooled=False),
                batch_size=BATCH_SIZE,
                shuffle=False,
                num_workers=0,
            )
            for i in range(NUM_CLIENTS)
        ]
# Function performing the evaluation
dict_cindex = evaluate_model_on_tests(model, test_dataloaders, metric)
print(dict_cindex)


/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))
  0%|          | 0/6 [00:00<?, ?it/s]/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))
100%|██████████| 6/6 [00:01<00:00,  5.31it/s]

{'client_test_0': 0.664804469273743, 'client_test_1': 0.5681818181818182, 'client_test_2': 0.7685185185185185, 'client_test_3': 0.5416666666666666, 'client_test_4': 0.7021276595744681, 'client_test_5': 1.0}


In [9]:
import torch
from flamby.utils import evaluate_model_on_tests

# 2 lines of code to change to switch to another dataset
from flamby.datasets.fed_tcga_brca import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    get_nb_max_rounds
)
from flamby.datasets.fed_tcga_brca import FedTcgaBrca as FedDataset

# 1st line of code to change to switch to another strategy
from flamby.strategies.fed_avg import FedAvg as strat

# We loop on all the clients of the distributed dataset and instantiate associated data loaders
train_dataloaders = [
            torch.utils.data.DataLoader(
                FedDataset(center = i, train = True, pooled = False),
                batch_size = BATCH_SIZE,
                shuffle = True,
                num_workers = 0
            )
            for i in range(NUM_CLIENTS)
        ]

lossfunc = BaselineLoss()
m = Baseline()

# Federated Learning loop
# 2nd line of code to change to switch to another strategy (feed the FL strategy the right HPs)
args = {
            "training_dataloaders": train_dataloaders,
            "model": m,
            "loss": lossfunc,
            "optimizer_class": torch.optim.SGD,
            "learning_rate": LR / 10.0,
            "num_updates": 100,
# This helper function returns the number of rounds necessary to perform approximately as many
# epochs on each local dataset as with the pooled training
            "nrounds": get_nb_max_rounds(100),
        }
s = strat(**args)
m = s.run()[0]

# Evaluation
# We only instantiate one test set in this particular case: the pooled one
test_dataloaders = [
            torch.utils.data.DataLoader(
                FedDataset(center=i, train = False, pooled = True),
                batch_size = BATCH_SIZE,
                shuffle = False,
                num_workers = 0,
            )
            for i in range(NUM_CLIENTS)
        ]
dict_cindex = evaluate_model_on_tests(m, test_dataloaders, metric)
print(dict_cindex)

  0%|          | 0/5 [00:00<?, ?it/s]/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))
  0%|          | 0/6 [00:00<?, ?it/s]/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))
100%|██████████| 6/6 [00:00<00:00, 19.45it/s]

{'client_test_0': 0.7688644688644689, 'client_test_1': 0.7688644688644689, 'client_test_2': 0.7688644688644689, 'client_test_3': 0.7688644688644689, 'client_test_4': 0.7688644688644689, 'client_test_5': 0.7688644688644689}
